## 单文件

In [ ]:
import requests
import os
import json
from IPython.display import display, JSON

def parse_single_pdf(file_path, api_url="http://127.0.0.1:8000/file_parse",
                     parse_method="auto", is_json_md_dump=True,
                     output_dir="output", return_layout=False,
                     return_info=False, return_content_list=False):
    """
    上传单个PDF文件到FastAPI服务进行解析，并保存md_content到本地文件

    参数:
        file_path (str): PDF文件的路径
        api_url (str): API端点URL
        parse_method (str): 解析方法，可选值：auto、ocr、txt
        is_json_md_dump (bool): 是否将解析结果写入.json和.md文件
        output_dir (str): 输出目录
        return_layout (bool): 是否返回解析后的PDF布局
        return_info (bool): 是否返回解析后的PDF信息
        return_content_list (bool): 是否返回解析后的PDF内容列表

    返回:
        dict: API响应结果
    """
    # 检查文件是否存在
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"文件不存在: {file_path}")

    # 确保输出目录存在
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
        print(f"创建输出目录: {output_dir}")

    # 准备文件和参数
    with open(file_path, 'rb') as f:
        files = {'file': (os.path.basename(file_path), f, 'application/pdf')}

        data = {
            'parse_method': parse_method,
            'is_json_md_dump': str(is_json_md_dump).lower(),
            'output_dir': output_dir,
            'return_layout': str(return_layout).lower(),
            'return_info': str(return_info).lower(),
            'return_content_list': str(return_content_list).lower()
        }

        # 发送请求
        print(f"正在上传文件: {os.path.basename(file_path)}...")
        response = requests.post(api_url, files=files, data=data)

    # 处理响应
    if response.status_code == 200:
        print("文件解析成功!")
        try:
            result = response.json()

            # 保存md_content到本地文件
            if 'md_content' in result:
                # 获取文件名（不含扩展名）
                base_name = os.path.splitext(os.path.basename(file_path))[0]
                md_file_path = os.path.join(output_dir, f"{base_name}.md")

                # 写入md文件
                with open(md_file_path, 'w', encoding='utf-8') as md_file:
                    md_file.write(result['md_content'])
                print(f"已保存Markdown文件: {md_file_path}")

            return result
        except json.JSONDecodeError:
            return response.text
    else:
        print(f"请求失败，状态码: {response.status_code}")
        print(f"错误信息: {response.text}")
        return None

if __name__ == "__main__":
    # 可配置参数
    file_path = r"D:\知识库\管局知识库\2024部应急及互联互通合同\2024部应急及互联互通合同\1-上会材料\局零余额上会签报20240322-定.pdf"  # 替换为您的PDF文件路径
    api_url = "http://127.0.0.1:8000/file_parse"
    parse_method = "auto"  # 可选: auto, ocr, txt
    is_json_md_dump = True  # 是否将结果保存为文件
    output_dir = r"D:\mineru_output"  # 输出目录
    return_layout = False  # 是否返回布局信息
    return_info = True  # 是否返回PDF信息
    return_content_list = True  # 是否返回内容列表

    # 调用函数
    result = parse_single_pdf(
        file_path=file_path,
        api_url=api_url,
        parse_method=parse_method,
        is_json_md_dump=is_json_md_dump,
        output_dir=output_dir,
        return_layout=return_layout,
        return_info=return_info,
        return_content_list=return_content_list
    )

    # 显示结果
    if result:
        if isinstance(result, dict):
            display(JSON(result))
        else:
            print(result)

正在上传文件: 局零余额上会签报20240322-定.pdf...
文件解析成功!


<IPython.core.display.JSON object>

## 多文件

In [ ]:
import requests
import os
import json
import time
from tqdm.notebook import tqdm
from IPython.display import display, JSON

def process_pdf_folder(folder_path, api_url="http://127.0.0.1:8000/file_parse",
                       parse_method="auto", is_json_md_dump=True,
                       return_layout=False, return_info=False,
                       return_content_list=False):
    """
    批量处理文件夹中的PDF文件，并保存md_content到本地文件

    参数:
        folder_path (str): 包含PDF文件的文件夹路径
        api_url (str): API端点URL
        parse_method (str): 解析方法，可选值：auto、ocr、txt
        is_json_md_dump (bool): 是否将解析结果写入.json和.md文件
        return_layout (bool): 是否返回解析后的PDF布局
        return_info (bool): 是否返回解析后的PDF信息
        return_content_list (bool): 是否返回解析后的PDF内容列表

    返回:
        dict: 处理结果统计
    """
    # 检查文件夹是否存在
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"文件夹不存在: {folder_path}")

    # 创建输出目录
    output_dir = os.path.join(folder_path, "processed")
    os.makedirs(output_dir, exist_ok=True)
    print(f"创建输出目录: {output_dir}")

    # 获取所有PDF文件
    pdf_files = [f for f in os.listdir(folder_path)
                if f.lower().endswith('.pdf') and os.path.isfile(os.path.join(folder_path, f))]

    if not pdf_files:
        print(f"在 {folder_path} 中没有找到PDF文件")
        return {"processed": 0, "success": 0, "failed": 0}

    # 处理统计
    stats = {"processed": 0, "success": 0, "failed": 0, "results": {}}

    # 使用tqdm创建进度条
    for pdf_file in tqdm(pdf_files, desc="处理PDF文件"):
        file_path = os.path.join(folder_path, pdf_file)

        try:
            # 准备文件和参数
            with open(file_path, 'rb') as f:
                files = {'file': (pdf_file, f, 'application/pdf')}

                data = {
                    'parse_method': parse_method,
                    'is_json_md_dump': str(is_json_md_dump).lower(),
                    'output_dir': output_dir,
                    'return_layout': str(return_layout).lower(),
                    'return_info': str(return_info).lower(),
                    'return_content_list': str(return_content_list).lower()
                }

                # 发送请求
                response = requests.post(api_url, files=files, data=data)

            # 处理响应
            stats["processed"] += 1

            if response.status_code == 200:
                stats["success"] += 1
                try:
                    result = response.json()
                    stats["results"][pdf_file] = {"status": "success", "data": result}

                    # 保存md_content到本地文件
                    if 'md_content' in result:
                        # 获取文件名（不含扩展名）
                        base_name = os.path.splitext(pdf_file)[0]
                        md_file_path = os.path.join(output_dir, f"{base_name}.md")

                        # 写入md文件
                        with open(md_file_path, 'w', encoding='utf-8') as md_file:
                            md_file.write(result['md_content'])
                        print(f"已保存Markdown文件: {md_file_path}")

                except json.JSONDecodeError:
                    stats["results"][pdf_file] = {"status": "success", "data": response.text}
            else:
                stats["failed"] += 1
                stats["results"][pdf_file] = {
                    "status": "failed",
                    "code": response.status_code,
                    "error": response.text
                }

            # 添加短暂延迟，避免请求过快
            time.sleep(0.5)

        except Exception as e:
            stats["processed"] += 1
            stats["failed"] += 1
            stats["results"][pdf_file] = {"status": "error", "error": str(e)}

    # 打印处理结果
    print(f"\n处理完成! 总计: {stats['processed']}, 成功: {stats['success']}, 失败: {stats['failed']}")
    print(f"处理结果保存在: {output_dir}")

    return stats

if __name__ == "__main__":
    # 可配置参数
    folder_path = r"D:\知识库\合同下载-ICT\test"  # 替换为您的PDF文件夹路径
    api_url = "http://127.0.0.1:8000/file_parse"
    parse_method = "auto"  # 可选: auto, ocr, txt
    is_json_md_dump = True  # 是否将结果保存为文件
    return_layout = False  # 是否返回布局信息
    return_info = True  # 是否返回PDF信息
    return_content_list = True  # 是否返回内容列表

    # 调用函数
    results = process_pdf_folder(
        folder_path=folder_path,
        api_url=api_url,
        parse_method=parse_method,
        is_json_md_dump=is_json_md_dump,
        return_layout=return_layout,
        return_info=return_info,
        return_content_list=return_content_list
    )

    # 显示成功处理的第一个文件结果（如果有）
    successful_files = [f for f, r in results["results"].items() if r["status"] == "success"]
    if successful_files:
        first_success = successful_files[0]
        print(f"\n显示第一个成功处理的文件 ({first_success}) 的结果:")
        result_data = results["results"][first_success]["data"]
        if isinstance(result_data, dict):
            display(JSON(result_data))
        else:
            print(result_data)

处理PDF文件:   0%|          | 0/4 [00:00<?, ?it/s]


处理完成! 总计: 4, 成功: 4, 失败: 0
处理结果保存在: D:\知识库\合同下载-ICT\test\processed

显示第一个成功处理的文件 (CMJS-WX-202400023+钱桥属地制造企业智慧安全用电监控服务二期项目前向合同（无锡拓驰仓储有限公司）.pdf) 的结果:


<IPython.core.display.JSON object>